# PA Quantum Neuron

Build a classical neuron with a quantum circuit

In [71]:
from pennylane import numpy as np
# pennylane code
import pennylane as qml
# from pennylane import numpy as np


## Basis encoding

In [78]:
# Basis encoding
x_1 = [5 ,6]
x_2 = [4 ,1]

# dec, binary, qstate
print('x_1 =>\n decimal: [{}, {}]'.format(x_1[0],x_1[1])
      , '\n binary: [{:03b}, {:03b}]'.format(x_1[0],x_1[1])
      , '\n basis encoded quantum state: |{:03b}{:03b}>'.format(x_1[0],x_1[1])
      , '\n' )

print('x_2 =>\n decimal: [{}, {}]'.format(x_2[0],x_2[1])
      , '\n binary: [{:03b}, {:03b}]'.format(x_2[0],x_2[1])
      , '\n basis encoded quantum state: |{:03b}{:03b}>'.format(x_2[0],x_2[1]) )

x_1 =>
 decimal: [5, 6] 
 binary: [101, 110] 
 basis encoded quantum state: |101110> 

x_2 =>
 decimal: [4, 1] 
 binary: [100, 001] 
 basis encoded quantum state: |100001>


In [73]:

# quantum device where you want to run and how many Qubits
dev = qml.device('default.qubit', wires=6)

@qml.qnode(dev)
def circuit(data):
    for i in range(6):
        qml.Hadamard(i)
    for i in range(len(data)):
        BasisEmbedding(features=data[i], wires=range(6),do_queue=True)
    return  qml.state()

data=[[1,0,1,1,1,0],
      [1,0,0,0,0,1]]

circuit(data)

print(circuit.draw(show_all_wires=True))

# Watch out when intepreting the drawing
# => The last X-gate on qubit #5 is should be on the right hand side (3rd gate position)
# pennylane drawings don't keep paddings


 0: ──H──X──X──╭┤ State 
 1: ──H────────├┤ State 
 2: ──H──X─────├┤ State 
 3: ──H──X─────├┤ State 
 4: ──H──X─────├┤ State 
 5: ──H──X─────╰┤ State 

